### Generation of LTLf formula + partition file for triangle-tireworld domain (non-deterministic). Amount of layers can be altered

In [ ]:
# give number of layers as input
number_of_layers = 3

def generate_states(layers):
    all_states = []
    
    # Generate states for each layer
    for layer in range(1, layers + 1):
        current_layer_states = [f'{i}{layer-i+1}' for i in range(1, layer + 1)]
        all_states.append(current_layer_states)    
    return all_states

def print_and_return_states(all_states):
    node_names = []
    for layer in all_states:
        print("layer: ", layer)
        for location in layer:
            node_names.append(location)
    return node_names

states = generate_states(number_of_layers)
node_names = print_and_return_states(states)

In [ ]:
def get_change_tire_actions(nodes):
    action_list = []
    for node in nodes:
        action_list.append(("changetire_"+ node))
    return action_list

def get_spare_tire_states(nodes):
    action_list = []
    for node in nodes:
        action_list.append(("sparein_"+ node))
    return action_list

def get_vehicle_at_states(nodes):
    action_list = []
    for node in nodes:
        action_list.append(("vehicleat_"+ node))
    return action_list

def get_paths_and_movecar(all_states):
    paths = []
    move_car_list = []
    
    for i, layer in enumerate(all_states):
        for j, location in enumerate(layer):
            # try to add path to right node
            if j+1< len(layer):
                paths.append("road_"+str(all_states[i][j])+"_"+str(all_states[i][j+1]))
            # try to add path to left node
            if j != 0:
                paths.append("road_"+str(all_states[i][j])+"_"+str(all_states[i][j-1]))
            # try to add path to top right node
            if i != 0:
                try:
                    paths.append("road_"+str(all_states[i][j])+"_"+str(all_states[i-1][j]))
                except: pass
                if j != 0:
                    # try to add path to top left node
                    try:
                        paths.append("road_"+str(all_states[i][j])+"_"+str(all_states[i-1][j-1]))
                    except: pass
            # try to add path to bottom right node
            if i != len(all_states):
                try:
                    paths.append("road_"+str(all_states[i][j])+"_"+str(all_states[i+1][j+1]))
                except: pass
            # try to add path to bottom left node
            try:
                paths.append("road_"+str(all_states[i][j])+"_"+str(all_states[i+1][j]))
            except: pass
            
    # IF ANY ROADS NEED TO BE REMOVED, CAN BE DONE IN LINE BELOW. Moves allowed by agent are added dynamically
    roads_to_remove = []
    
    paths = [road for road in paths if road not in roads_to_remove]
    
    for path in paths:
        action, from_l, to_l = path.split('_')
        move_car_list.append("movecar_"+from_l+"_"+to_l)
            
    return paths, move_car_list

In [ ]:
print("environment locations: ", node_names)
print()
print("change tire actions: ", get_change_tire_actions(node_names))
print()
print("spare tires: ", get_spare_tire_states(node_names))
print()
print("vehicle at: ", get_vehicle_at_states(node_names))
print()
paths, move_car_list = get_paths_and_movecar(states)
print("roads: ", paths)
print()
print("move car options: ", move_car_list)
print()
flattire = "flattire"
print("flattire: ", flattire)
print()

In [ ]:
def get_mutual_exclusion_agent_actions(agent_actions):
    all_actions_or = " | ".join(agent_actions)
    action_combinations = []
    
    for i, action in enumerate(agent_actions):
        for j in range(i+1, len(agent_actions)):
            action_combinations.append("!(" + agent_actions[i] + " & " + agent_actions[j] + ")")
    
    action_comb_str = " & ".join(action_combinations)
    
    return all_actions_or, action_comb_str

def get_agent_preconditions():
    prec_list = []
    # preconditions for 'movecar'
    for move in move_car_list:
        action, from_l, to_l = move.split('_')
        prec_list.append("(!(vehicleat_" + from_l + " & road_" + from_l + "_" + to_l + " & !flattire) -> X(!" + move + "))")
        
    # TODO: check if its not necessary to have a flat tire before changing it
    # preconditions for 'changetire'
    for change_act in get_change_tire_actions(node_names):
        action, from_l = change_act.split('_')
        prec_list.append("(!(sparein_" + from_l + " & vehicleat_" + from_l + " & flattire) -> X(!" + change_act + "))")
    
    return prec_list

In [ ]:
def get_environment_transitions():
    effect_list = []
    # effect 'movecar'
    for move in move_car_list:
        action, from_l, to_l = move.split('_')
        effect_list.append("(vehicleat_" + from_l + " -> X(" + move + " -> (vehicleat_" + to_l + " & (flattire | !flattire))))")
        
    # effect 'changetire'
    for change_act in get_change_tire_actions(node_names):
        action, from_l = change_act.split('_')
        effect_list.append("(sparein_" + from_l + " & vehicleat_" + from_l + " -> X(" + change_act  + " -> (!sparein_" + from_l + " & !flattire & vehicleat_" + from_l+ ")))")
    
    return effect_list

In [ ]:
# use this for specifying which fluents never change during a run (like roads)
def get_consistent_fluents(fluents):
    return "G(" + " & ".join(fluents) + ")"

In [ ]:
def get_next_spare_status():
    sparetires = get_spare_tire_states(node_names)
    sparenext = []
    
    for spare in sparetires:
        action, location = spare.split('_')
        sparenext.append("(!" + spare + " -> X(!" + spare + "))")
        sparenext.append("((" + spare + " & !changetire_" + location + ") -> X(" + spare + "))")
    
    return "G(" + " & ".join(sparenext) + ")"

def get_agent_environment_actions():
    paths, move_car_list = get_paths_and_movecar(states)
    
    environment_actions = get_vehicle_at_states(node_names) + get_spare_tire_states(node_names) + paths
    agent_actions = move_car_list + get_change_tire_actions(node_names)
    
    return environment_actions, agent_actions

### the goal and initialization have to be set by hand (these are hardcoded):

In [ ]:
# this has to be set by hand
def get_goal():
    return "F(vehicleat_21)"

# this has to be set by hand
def get_initialization():
    init_list = ["!flattire", "vehicleat_11", "sparein_12", "sparein_11", "sparein_21"]
    
    return " & ".join(init_list)

### code block below retrieves the mutual exclusion of agent actions and specific environment fluents. Also agent preconditions are retrieved:

In [ ]:
# 'mutual_excl_all' contains the or-statement between all actions
# 'mutual_excl_comb' contains the negation of all possible agent action combinations
env_act_list, agent_act_list = get_agent_environment_actions()

mutual_excl_all, mutual_excl_comb = get_mutual_exclusion_agent_actions(agent_act_list)
mutual_excl_all_env, mutual_excl_comb_env = get_mutual_exclusion_agent_actions(get_vehicle_at_states(node_names))

agent_prec_list = get_agent_preconditions()

### partition file (copy and paste in '.part'-file)

In [ ]:
# retrieve all agent + environment actions in separate lists

# print out partition file
print(".inputs: ", " ".join(env_act_list), " flattire", sep='')
print(".outputs: ", " ".join(agent_act_list), sep='')

### LTLf formula (copy and paste in '.ltlf'-file):

In [ ]:
# init
print("(", get_initialization(), ")", sep='')

In [ ]:
# agent
print("((", mutual_excl_all, ") & ", mutual_excl_comb, ")", sep='')

In [ ]:
# environment
# TODO: add flattire status
print("(((", mutual_excl_all_env, ") & ", mutual_excl_comb_env, ") & (", " & ".join(get_environment_transitions()), ") & (", get_next_spare_status(), ") & ", get_consistent_fluents(paths), ")", sep='')

In [ ]:
# preconditions agent actions
print("(X(", env_act_list[0], " | !", env_act_list[0] , ")) -> ("," & ".join(get_agent_preconditions()),")", sep="")

In [ ]:
# goal
print(get_goal(), sep='')

In [ ]:
# complete formula, written as follows: (G(phi_agent_unique_act) & ((phi_env_init & G(phi_env_transitions_and_fluents)) -> ((G(phi_agent_act_preconditions)) & (phi_goal))))
print("(G((", mutual_excl_all, ") & ", mutual_excl_comb, ")) & (((", get_initialization(), ") & G(((", mutual_excl_all_env, ") & ", mutual_excl_comb_env, ") & (", " & ".join(get_environment_transitions()), ") & (", get_next_spare_status(), ") & ", get_consistent_fluents(paths), ")) -> ((G((X(", env_act_list[0], " | !", env_act_list[0] , ")) -> ("," & ".join(get_agent_preconditions()),"))) & (", get_goal(), ")))" , sep='')